In [1]:
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from counterfactual_functions import *
from helper import *
import pyipopt
from autograd import grad, hessian



In [2]:
username='boraozaltun'

data_dict = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_20200226.pickle')

# For stability
print(np.where(data_dict['phi_R'] == 0))
#data_dict['phi_R'][data_dict['phi_R'] == 0 ] = 1e-14



(array([  0,   0,   0, ..., 133, 133, 133]), array([18, 19, 20, ..., 23, 24, 25]))


In [3]:
reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

In [4]:
n = data_dict['n']
g = data_dict['g']
k = data_dict['k']

data_dict['R_hat'] = np.ones((n, g))
X_0 = np.ones(n*g + n)*1.0

In [5]:
F = lambda x: reduced_counterfactual(x, data_dict)
f = lambda x: np.linalg.norm(reduced_counterfactual(x, data_dict))


In [6]:
f(X_0) # This should be just about 1...

1.5797580287425462e-14

In [7]:
X = X_0
data = dict(data_dict)
r_hat, w_hat = get_values_from_X_reduced(X, data)
# print(r_hat.mean())

C_g_hat = get_C_g_hat(w_hat, r_hat, data)
E_hat = get_E_hat(w_hat, r_hat, data)

P_g_goods_hat = get_P_g_goods_hat(C_g_hat, data)

C_k_hat = get_C_k_hat(w_hat, P_g_goods_hat, data)

P_k_goods_hat = get_P_k_goods_hat(C_k_hat, data)

P_k_hat = get_P_k_hat(P_k_goods_hat, data)

D_k_hat = get_D_k_hat(E_hat, P_k_goods_hat, P_k_hat, data)

Y_k_hat = get_Y_k_hat(C_k_hat, P_k_goods_hat, D_k_hat, data)
# print(np.mean(Y_k_hat))
D_g_hat = get_D_g_hat(P_g_goods_hat, C_k_hat, Y_k_hat, data)
# print(np.mean(D_g_hat))
Y_g_hat = get_Y_g_hat(C_g_hat, P_g_goods_hat, D_g_hat, data)

In [8]:
data = dict(data_dict)
part1 = data['phi_R'] * (r_hat ** (1- data['rho_g'].reshape((1, data['g']))))
part2 = data['phi_L_g'] * (w_hat.reshape((data['n'], 1)) ** (1 - data['rho_g'].reshape((1, data['g']))) )

C_g_hat = (part1 + part2) ** (1/(1-data['rho_g'].reshape((1, data['g'])) ))


In [9]:
np.linalg.norm(C_g_hat - 1)

0.0

In [10]:
np.linalg.norm(data['phi_R']+data['phi_L_g'] -1)

0.0

In [11]:
np.linalg.norm(r_hat - get_r_hat(r_hat, Y_g_hat, C_g_hat, data))

1.5634053252502143e-14

In [12]:
np.linalg.norm(w_hat - get_w_hat(w_hat, Y_g_hat, Y_k_hat, C_k_hat, C_g_hat, data))

2.2671395712616936e-15

In [13]:
part1_part1 = data['phi_L_g']*data['Y_g']/(data['E']*data['e_L']).reshape((data['n'], 1))
part1_part2 = ((w_hat.reshape((data['n'], 1))/C_g_hat)**(1-data['rho_g'].reshape((1, data['g'])))) *Y_g_hat

part1 = (part1_part1*part1_part2).sum(axis=1).reshape((data['n'], 1))

part2_part1 = data['phi_L_k']*data['Y_k']/(data['E']*data['e_L']).reshape((data['n'], 1))

part2_part2 =  ((w_hat.reshape((data['n'], 1))/C_k_hat)**(1-data['eta'].reshape((1, data['k'])))) *Y_k_hat

part2 = (part2_part1*part2_part2).sum(axis=1).reshape((data['n'], 1))

w_hat_new = ((part1 + part2)/(part1_part1.sum(axis=1)+ part2_part1.sum(axis=1)).reshape((data['n'], 1)))

In [14]:
np.linalg.norm(w_hat-w_hat_new)

2.2315206618374916e-15